<a href="https://colab.research.google.com/github/Laurindocak/Calculo_Metricas/blob/main/YOLOv8_cachorro_gato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# YOLOv8 — Treino com **cachorro** e **gato** (Transfer Learning, Colab)
Notebook completo para:
1. Preparar dataset no formato YOLO (imagens e rótulos).
2. (Opcional) Converter **LabelMe JSON → YOLO**.
3. Treinar, avaliar e fazer inferência usando **Ultralytics YOLOv8**.
4. Exportar pesos e resultados.

> **Classes definidas:** `cachorro`, `gato`


In [11]:

#@title Instalar dependências (Ultralytics) { display-mode: "form" }
# Se estiver no Colab, a linha abaixo fará a instalação.
!pip -q install ultralytics>=8.2.0
from ultralytics import YOLO
print("Ultralytics importado com sucesso.")


Ultralytics importado com sucesso.


In [12]:

#@title Conectar ao Google Drive (Colab) { display-mode: "form" }
# Execute esta célula no Google Colab para montar o Drive. Se estiver em ambiente local, pode ignorar.
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    IN_COLAB = True
except Exception as e:
    print("Parece que não estamos no Colab ou o Drive não foi montado:", e)
    IN_COLAB = False


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:

#@title Definir caminhos do projeto e criar estrutura { display-mode: "form" }
from pathlib import Path
import os, textwrap, yaml

# Altere se desejar outro caminho dentro do Drive
BASE_DIR = Path('/content/drive/MyDrive/YOLO_cachorro_gato') if 'IN_COLAB' in globals() and IN_COLAB else Path('/content/YOLO_cachorro_gato')
DATASET_DIR = BASE_DIR / 'datasets' / 'cachorro_gato'
IMAGES_TRAIN = DATASET_DIR / 'images' / 'train'
IMAGES_VAL   = DATASET_DIR / 'images' / 'val'
IMAGES_TEST  = DATASET_DIR / 'images' / 'test'
LABELS_TRAIN = DATASET_DIR / 'labels' / 'train'
LABELS_VAL   = DATASET_DIR / 'labels' / 'val'
LABELS_TEST  = DATASET_DIR / 'labels' / 'test'
RESULTS_DIR  = BASE_DIR / 'results'

for p in [IMAGES_TRAIN, IMAGES_VAL, IMAGES_TEST, LABELS_TRAIN, LABELS_VAL, LABELS_TEST, RESULTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Estrutura criada em:", BASE_DIR)
for p in [DATASET_DIR, IMAGES_TRAIN, IMAGES_VAL, IMAGES_TEST, LABELS_TRAIN, LABELS_VAL, LABELS_TEST]:
    print("-", p)

# Gerar data.yaml automaticamente
data_yaml = {
    'path': str(DATASET_DIR),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': ['cachorro', 'gato']
}
with open(DATASET_DIR / 'data.yaml', 'w') as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False, allow_unicode=True)

print("\nArquivo data.yaml salvo em:", DATASET_DIR / 'data.yaml')
print("Conteúdo:")
print(yaml.safe_dump(data_yaml, sort_keys=False, allow_unicode=True))


Estrutura criada em: /content/drive/MyDrive/YOLO_cachorro_gato
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/images/train
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/images/val
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/images/test
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/labels/train
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/labels/val
- /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/labels/test

Arquivo data.yaml salvo em: /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/data.yaml
Conteúdo:
path: /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato
train: images/train
val: images/val
test: images/test
names:
- cachorro
- gato




## 📥 Como colocar as imagens e rótulos
- Coloque suas imagens `.jpg/.png` dentro de:
  - `datasets/cachorro_gato/images/train`
  - `datasets/cachorro_gato/images/val`
  - `datasets/cachorro_gato/images/test` *(opcional)*
- Os rótulos YOLO `.txt` correspondentes devem ir em:
  - `datasets/cachorro_gato/labels/train`
  - `datasets/cachorro_gato/labels/val`
  - `datasets/cachorro_gato/labels/test` *(opcional)*

> **Formato YOLO por linha**: `class_id x_center y_center width height` (valores normalizados entre 0 e 1).
>
> **IDs de classe** neste projeto: `0 = cachorro`, `1 = gato`.



## 🔁 (Opcional) Converter rótulos do **LabelMe JSON** para YOLO
- Coloque seus arquivos `.json` do LabelMe e as imagens correspondentes numa pasta temporária.
- A célula abaixo converte cada JSON para um `.txt` no formato YOLO e salva em `labels/train` (ou `labels/val` conforme ajustar).


In [14]:

#@title Conversor LabelMe JSON → YOLO (caixas) { display-mode: "form" }
import json, shutil
from PIL import Image
from pathlib import Path

def bbox_from_shape(shape):
    # Suporta shapes tipo rectangle ou polygon (usa bounding box do polígono)
    if shape.get('shape_type', 'polygon') == 'rectangle':
        # rectangle: points = [[x1, y1], [x2, y2]]
        (x1, y1), (x2, y2) = shape['points']
        xmin, ymin = min(x1, x2), min(y1, y2)
        xmax, ymax = max(x1, x2), max(y1, y2)
    else:
        xs = [p[0] for p in shape['points']]
        ys = [p[1] for p in shape['points']]
        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)
    return xmin, ymin, xmax, ymax

def convert_labelme_folder_to_yolo(json_folder, images_out_dir, labels_out_dir, class_map):
    json_folder = Path(json_folder)
    images_out  = Path(images_out_dir); images_out.mkdir(parents=True, exist_ok=True)
    labels_out  = Path(labels_out_dir); labels_out.mkdir(parents=True, exist_ok=True)

    for json_path in json_folder.glob("*.json"):
        with open(json_path, 'r') as f:
            data = json.load(f)

        image_path = json_folder / data['imagePath']
        if not image_path.exists():
            # tenta localizar a imagem por nome na pasta
            possible = list(json_folder.glob(Path(data['imagePath']).name))
            if possible:
                image_path = possible[0]
            else:
                print(f"[AVISO] Imagem não encontrada para {json_path.name}")
                continue

        # abre imagem para pegar dimensões
        with Image.open(image_path) as img:
            w, h = img.size

        # copia imagem para pasta de destino
        dest_img = images_out / image_path.name
        if not dest_img.exists():
            shutil.copy2(image_path, dest_img)

        # cria label .txt
        label_path = labels_out / (image_path.stem + ".txt")
        lines = []
        for shape in data.get('shapes', []):
            label = shape.get('label', '').strip().lower()
            if label not in class_map:
                # tenta normalizar plurais/acentos manualmente se necessário
                continue
            cls_id = class_map[label]
            xmin, ymin, xmax, ymax = bbox_from_shape(shape)
            # converte para YOLO (normalizado)
            x_c = ((xmin + xmax) / 2.0) / w
            y_c = ((ymin + ymax) / 2.0) / h
            ww  = (xmax - xmin) / w
            hh  = (ymax - ymin) / h
            # clip para [0,1]
            x_c = max(0, min(1, x_c)); y_c = max(0, min(1, y_c))
            ww  = max(0, min(1, ww));  hh  = max(0, min(1, hh))
            lines.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {ww:.6f} {hh:.6f}")

        with open(label_path, 'w') as lf:
            lf.write("\n".join(lines))

        print("Convertido:", json_path.name, "→", label_path.name)

# Exemplo de uso:
# Ajuste as pastas e se quer mandar para train/val
# class_map mapeia os rótulos do LabelMe (em minúsculas) para IDs YOLO
example_input_folder = "/content/labelme_jsons"  #@param {type:"string"}
send_to_split = "train"  #@param ["train", "val"]
images_out = str(IMAGES_TRAIN if send_to_split == "train" else IMAGES_VAL)
labels_out = str(LABELS_TRAIN if send_to_split == "train" else LABELS_VAL)

# Mapeie os rótulos do LabelMe para os IDs das classes deste projeto
class_map = {"cachorro": 0, "gato": 1}

# Para rodar, descomente a linha abaixo e garanta que `example_input_folder` existe com JSONs + imagens
# convert_labelme_folder_to_yolo(example_input_folder, images_out, labels_out, class_map)
print("Pronto. Ajuste as variáveis e descomente a chamada para converter.")


Pronto. Ajuste as variáveis e descomente a chamada para converter.



## 🪄 (Opcional) Criar split **train/val** automaticamente
Se você tem todas as imagens em uma única pasta e já possui rótulos YOLO correspondentes (`.txt` com mesmo nome),
esta célula divide em treino/validação.


In [15]:

#@title Criar split aleatório train/val a partir de uma pasta única { display-mode: "form" }
import random, shutil
from pathlib import Path

single_images_dir = "/content/minhas_imagens"  #@param {type:"string"}
single_labels_dir = "/content/meus_rotulos"   #@param {type:"string"}
val_ratio = 0.2                                #@param {type:"number"}

def move_pair(img_path, dst_img_dir, dst_lbl_dir):
    lbl_path = Path(single_labels_dir) / (img_path.stem + ".txt")
    shutil.copy2(img_path, Path(dst_img_dir) / img_path.name)
    if lbl_path.exists():
        shutil.copy2(lbl_path, Path(dst_lbl_dir) / lbl_path.name)

if False:  # Mude para True para executar a divisão
    imgs = [p for p in Path(single_images_dir).glob("*.*") if p.suffix.lower() in {".jpg", ".jpeg", ".png"}]
    random.shuffle(imgs)
    n_val = int(len(imgs) * val_ratio)
    val_set = set(imgs[:n_val])
    for img in imgs:
        if img in val_set:
            move_pair(img, IMAGES_VAL, LABELS_VAL)
        else:
            move_pair(img, IMAGES_TRAIN, LABELS_TRAIN)
    print("Split concluído!")
else:
    print("Defina o bloco para True para rodar a divisão.")


Defina o bloco para True para rodar a divisão.



## 🏋️‍♀️ Treinar YOLOv8 (transfer learning)
- Usa pesos base (`yolov8n.pt`) e treina nas classes `cachorro` e `gato`.
- Ajuste `epochs`, `imgsz`, `batch`, `lr0` conforme seu hardware/dataset.


In [16]:

#@title Iniciar o treino { display-mode: "form" }
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # troque por 'yolov8s.pt'/'m'/'l' se tiver GPU melhor
results = model.train(
    data=str(DATASET_DIR / 'data.yaml'),
    epochs=50,     # ajuste
    imgsz=640,     # ajuste
    batch=16,      # ajuste
    lr0=0.01,      # ajuste
    workers=2,     # Colab costuma limitar workers
    project=str(RESULTS_DIR),
    name='yolov8n_cachorro_gato',
    verbose=True
)
print("Treino concluído. Resultados em:", RESULTS_DIR / 'yolov8n_cachorro_gato')


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_cachorro_gato3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=T

FileNotFoundError: [34m[1mtrain: [0mError loading data from /content/drive/MyDrive/YOLO_cachorro_gato/datasets/cachorro_gato/images/train
See https://docs.ultralytics.com/datasets for dataset formatting guidance.


## 📊 Avaliação e métricas
Gera **mAP**, **precision**, **recall**, **F1** e matriz de confusão.


In [ ]:

#@title Validação / Métricas { display-mode: "form" }
val_results = model.val()
print(val_results)

# Mostrar caminho dos artefatos (curvas, confusão, PR, etc.)
from pathlib import Path
run_dir = Path(RESULTS_DIR) / 'yolov8n_cachorro_gato'
print("Arquivos de gráficos e métricas (se gerados):", list(run_dir.glob("**/*"))[:10], "...")



## 🔎 Inferência (previsões) em imagens de teste
- Coloque algumas imagens em `images/test/` ou aponte `source` para outra pasta (ou vídeo).
- As imagens com detecções serão salvas com bounding boxes na pasta de `predict` da execução.


In [ ]:

#@title Rodar previsões e salvar saídas { display-mode: "form" }
pred = model.predict(
    source=str(IMAGES_TEST) if any(IMAGES_TEST.iterdir()) else str(IMAGES_VAL),
    save=True,
    conf=0.25
)
print("Predições salvas em:", pred[0].save_dir if pred else "Verifique se há imagens de entrada.")



## 📦 Exportar pesos e artefatos
Salva pesos finais e zippa a pasta de resultados para download/compartilhamento.


In [ ]:

#@title Exportar pesos e compactar resultados { display-mode: "form" }
best_pt = RESULTS_DIR / 'yolov8n_cachorro_gato' / 'weights' / 'best.pt'
last_pt = RESULTS_DIR / 'yolov8n_cachorro_gato' / 'weights' / 'last.pt'
print("Pesos esperados:", best_pt, last_pt)

# Compactar pasta de resultados
import shutil, os
zip_out = str(RESULTS_DIR) + ".zip"
if os.path.exists(zip_out):
    os.remove(zip_out)
shutil.make_archive(str(RESULTS_DIR), 'zip', root_dir=RESULTS_DIR)
print("ZIP criado em:", zip_out)



## 🛠️ Dicas & Troubleshooting
- **Dataset vazio**: confirme que existem imagens e rótulos correspondentes.
- **IDs de classe incorretos**: revise o `data.yaml` e os `.txt` (0=cachorro, 1=gato).
- **Overfitting**: reduza `epochs`, aumente `val_ratio`, adicione **augmentation** (parâmetros em `model.train()` ou via Ultralytics settings).
- **Baixa mAP em uma classe**: colete mais dados dessa classe, melhore diversidade e rotulagem.
- **GPU no Colab**: `Runtime > Change runtime type > T4/A100` (quando disponível).
